In [20]:
import numpy as np
import pandas as pd
import random
import string
import tensorflow as tf
from tensorflow import keras
import tensorboard
import os
from pathlib import Path
import datetime

from sklearn.model_selection import ParameterSampler
from sklearn.model_selection import train_test_split

print('TensorFlow version: ', tf.__version__)
print('Keras version: ', keras.__version__)
print('Tensorboard version:', tensorboard.__version__)



TensorFlow version:  2.0.0
Keras version:  2.2.4-tf
Tensorboard version: 2.0.0


In [21]:
data_path = Path('/home/tim/Documents/flask_experiment/data')

file_csv = []
index_count = 0
for file in os.listdir(data_path):
    # open up the .csv file
    if file.endswith(".csv"):
        file_csv.append(file)
        print(file)
        df = pd.read_csv(data_path / file,)
        # scale some of the large values
        df['age'] = df['age']/100.0
        df['dist'] = df['dist']/25000.0
        df['year'] = df['year']/2020.0
        df = df.drop(['model'],axis=1,inplace=False)
        if index_count == 0:
            y = df['quote'].to_numpy()
            print(y.shape)
            df = df.drop(['quote'],axis=1,inplace=False)
            df = pd.get_dummies(df,drop_first=True)
            col_list = list(df.columns)
            X = df.to_numpy()
            index_count += 1
        else:
            a = df['quote'].to_numpy()
            y = np.vstack((a, y))
            df = df.drop(['quote'],axis=1,inplace=False)
            df = pd.get_dummies(df,drop_first=True)
            X = np.dstack((df.to_numpy(), X))
            index_count += 1

X = np.reshape(X,(X.shape[0],-1,X.shape[1]))
print(X.shape)
print(y.shape)
print(file_csv)

company_24.csv
(20000,)
(20000, 1, 114)
(20000,)
['company_24.csv']


In [22]:
df.head()

,age,dist,year,city_Airdrie,city_Ajax,city_Aurora,city_Barrie,city_Belleville,city_Blainville,city_Brampton,...,make_bmw,make_chevy,make_ford,make_gmc,make_honda,make_mercedes,make_nissan,make_tesla,make_toyota,new_used_used
0,0.23,0.56520,0.992079,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
1,0.48,0.91680,0.995545,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
2,0.32,0.65368,0.999010,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
3,0.58,0.19180,0.992079,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
4,0.52,0.37404,0.998020,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1


In [23]:
X.shape

(20000, 1, 114)

In [24]:
file_csv[0].split('.')[0]

'company_24'

In [25]:
# split up the X and y for both the quote prediction and confidence interval and 
X_quote = X[0:int(X.shape[0]/2)]
y_quote = y[0:int(y.shape[0]/2)]
print(X_quote.shape)
print(y_quote.shape)
X_conf = X[int(X.shape[0]/2):]
y_conf = y[int(y.shape[0]/2):]
print(X_conf.shape)
print(y_conf.shape)

X_train_q, X_test_q, y_train_q, y_test_q = train_test_split(X_quote, y_quote, test_size=0.33, random_state=42)
X_train_q, X_val_q, y_train_q, y_val_q = train_test_split(X_train_q, y_train_q, test_size=0.33, random_state=42)

print("X_train_q shape:", X_train_q.shape, "\ty_train_q shape:", y_train_q.shape)
print("X_val_q:", X_val_q.shape, "\t\t\ty_val_q shape:", y_val_q.shape)
print("X_test_q:", X_test_q.shape, "\t\ty_test_q shape:", y_test_q.shape)



(10000, 1, 114)
(10000,)
(10000, 1, 114)
(10000,)
X_train_q shape: (4489, 1, 114) 	y_train_q shape: (4489,)
X_val_q: (2211, 1, 114) 			y_val_q shape: (2211,)
X_test_q: (3300, 1, 114) 		y_test_q shape: (3300,)


In [26]:
model = keras.models.Sequential([
    keras.layers.Dense(200, activation="relu", input_shape=[1, 114]),
    keras.layers.Dense(200, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation="linear")
])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1, 200)            23000     
_________________________________________________________________
dense_1 (Dense)              (None, 1, 200)            40200     
_________________________________________________________________
dropout (Dropout)            (None, 1, 200)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 1, 50)             10050     
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 50)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 1, 1)              51        
Total params: 73,301
Trainable params: 73,301
Non-trainable params: 0
____________________________________________________

In [6]:
model.compile(loss="mse",
              optimizer="adam",
              metrics=["mse"])

# create a name for the model so that we can track it in tensorboard
log_dir="logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "_quote"

# create tensorboard callback
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0,
                                                      update_freq='epoch',profile_batch=0)

earlystop_callback = tf.keras.callbacks.EarlyStopping(
        monitor="mse",
        patience=30,
        restore_best_weights=True,
        verbose=1,)

In [18]:
history = model.fit(X_train_q, y_train_q, 
                    epochs=200,
                    verbose=1,
                    validation_data=(X_val_q,y_val_q),
                    callbacks=[tensorboard_callback, earlystop_callback])

Train on 4489 samples, validate on 2211 samples
Epoch 1/200
4489/4489 [==============================] - 1s 268us/sample - loss: 1271.1785 - mse: 1271.1783 - val_loss: 437.3573 - val_mse: 437.3574
Epoch 2/200
4489/4489 [==============================] - 1s 116us/sample - loss: 1250.6368 - mse: 1250.6370 - val_loss: 323.1503 - val_mse: 323.1503
Epoch 3/200
4489/4489 [==============================] - 0s 105us/sample - loss: 1200.1515 - mse: 1200.1510 - val_loss: 263.7770 - val_mse: 263.7770
Epoch 4/200
4489/4489 [==============================] - 0s 108us/sample - loss: 1152.6072 - mse: 1152.6069 - val_loss: 269.7924 - val_mse: 269.7924
Epoch 5/200
4489/4489 [==============================] - 0s 107us/sample - loss: 1196.6953 - mse: 1196.6958 - val_loss: 275.3452 - val_mse: 275.3452
Epoch 6/200
4489/4489 [==============================] - 0s 106us/sample - loss: 1211.4388 - mse: 1211.4388 - val_loss: 307.4712 - val_mse: 307.4712
Epoch 7/200
4489/4489 [==============================] - 0

Epoch 110/200
4489/4489 [==============================] - 0s 100us/sample - loss: 1045.6539 - mse: 1045.6539 - val_loss: 263.6012 - val_mse: 263.6013
Epoch 111/200
4489/4489 [==============================] - 0s 100us/sample - loss: 1040.9005 - mse: 1040.9003 - val_loss: 370.3228 - val_mse: 370.3228
Epoch 112/200
4489/4489 [==============================] - 0s 105us/sample - loss: 1003.4643 - mse: 1003.4644 - val_loss: 356.1061 - val_mse: 356.1061
Epoch 113/200
4489/4489 [==============================] - 0s 99us/sample - loss: 1091.1166 - mse: 1091.1166 - val_loss: 388.7835 - val_mse: 388.7835
Epoch 114/200
4489/4489 [==============================] - 0s 102us/sample - loss: 1026.1380 - mse: 1026.1377 - val_loss: 262.5911 - val_mse: 262.5911
Epoch 115/200
4489/4489 [==============================] - 0s 100us/sample - loss: 1107.4792 - mse: 1107.4792 - val_loss: 264.6609 - val_mse: 264.6609
Epoch 116/200
4489/4489 [==============================] - 0s 100us/sample - loss: 1069.3939 - 

In [11]:
model.save('my_model.h5') 

In [12]:
y_predict_conf = model.predict(X_conf)

In [13]:
y_predict_conf = np.reshape(y_predict_conf,(y_predict_conf.shape[0],))
y_predict_conf.shape

(10000,)

In [14]:
y_predict_conf

array([ 0.08854777, -0.00409351, -0.13561016, ...,  0.02362314,
       -0.07776155, -0.05132986], dtype=float32)

In [19]:
y_conf_error = np.divide(np.abs(y_conf - y_predict_conf),y_conf)
y_conf_error

NameError: name 'y_predict_conf' is not defined

In [18]:
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_conf, y_conf_error, test_size=0.33, random_state=42)
X_train_c, X_val_c, y_train_c, y_val_c = train_test_split(X_train_c, y_train_c, test_size=0.33, random_state=42)

NameError: name 'y_conf_error' is not defined

In [46]:
model_conf = keras.models.Sequential([
    keras.layers.Dense(200, activation="relu", input_shape=[1, 114]),
    keras.layers.Dense(200, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation="sigmoid")
])

model_conf.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 1, 200)            23000     
_________________________________________________________________
dense_13 (Dense)             (None, 1, 200)            40200     
_________________________________________________________________
dropout_4 (Dropout)          (None, 1, 200)            0         
_________________________________________________________________
dense_14 (Dense)             (None, 1, 50)             10050     
_________________________________________________________________
dropout_5 (Dropout)          (None, 1, 50)             0         
_________________________________________________________________
dense_15 (Dense)             (None, 1, 1)              51        
Total params: 73,301
Trainable params: 73,301
Non-trainable params: 0
__________________________________________________

In [47]:
model_conf.compile(loss="mse",
              optimizer="adam",
              metrics=["mse"])

# create a name for the model so that we can track it in tensorboard
log_dir="logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "_conf"

# create tensorboard callback
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0,
                                                      update_freq='epoch',profile_batch=0)

earlystop_callback = tf.keras.callbacks.EarlyStopping(
        monitor="mse",
        patience=30,
        restore_best_weights=True,
        verbose=1,)

In [48]:
history = model_conf.fit(X_train_c, y_train_c, 
                    epochs=200,
                    verbose=1,
                    validation_data=(X_val_c,y_val_c),
                    callbacks=[tensorboard_callback, earlystop_callback])

Train on 4489 samples, validate on 2211 samples
Epoch 1/200
4489/4489 [==============================] - 1s 264us/sample - loss: 0.0151 - mse: 0.0151 - val_loss: 0.0098 - val_mse: 0.0098
Epoch 2/200
4489/4489 [==============================] - 0s 105us/sample - loss: 0.0092 - mse: 0.0092 - val_loss: 0.0087 - val_mse: 0.0087
Epoch 3/200
4489/4489 [==============================] - 0s 108us/sample - loss: 0.0082 - mse: 0.0082 - val_loss: 0.0089 - val_mse: 0.0089
Epoch 4/200
4489/4489 [==============================] - 0s 107us/sample - loss: 0.0073 - mse: 0.0073 - val_loss: 0.0082 - val_mse: 0.0082
Epoch 5/200
4489/4489 [==============================] - 0s 110us/sample - loss: 0.0069 - mse: 0.0069 - val_loss: 0.0077 - val_mse: 0.0077
Epoch 6/200
4489/4489 [==============================] - 0s 107us/sample - loss: 0.0062 - mse: 0.0062 - val_loss: 0.0077 - val_mse: 0.0077
Epoch 7/200
4489/4489 [==============================] - 0s 109us/sample - loss: 0.0059 - mse: 0.0059 - val_loss: 0.00

4489/4489 [==============================] - 0s 111us/sample - loss: 0.0010 - mse: 0.0010 - val_loss: 0.0082 - val_mse: 0.0082
Epoch 118/200
4489/4489 [==============================] - 1s 120us/sample - loss: 0.0011 - mse: 0.0011 - val_loss: 0.0085 - val_mse: 0.0085
Epoch 119/200
4489/4489 [==============================] - 1s 137us/sample - loss: 9.2124e-04 - mse: 9.2124e-04 - val_loss: 0.0080 - val_mse: 0.0080
Epoch 120/200
4489/4489 [==============================] - 1s 115us/sample - loss: 8.8561e-04 - mse: 8.8561e-04 - val_loss: 0.0084 - val_mse: 0.0084
Epoch 121/200
4489/4489 [==============================] - 1s 114us/sample - loss: 0.0010 - mse: 0.0010 - val_loss: 0.0083 - val_mse: 0.0083
Epoch 122/200
4489/4489 [==============================] - 1s 118us/sample - loss: 9.2713e-04 - mse: 9.2713e-04 - val_loss: 0.0083 - val_mse: 0.0083
Epoch 123/200
4489/4489 [==============================] - 1s 136us/sample - loss: 9.5377e-04 - mse: 9.5377e-04 - val_loss: 0.0085 - val_mse: 0.

In [49]:
model_conf.save('my_model_conf.h5') 

In [16]:
# load model
new_model = tf.keras.models.load_model('my_model_conf.h5')


In [17]:
X_test_c[0]

NameError: name 'X_test_c' is not defined

In [60]:
X_test_c[0].shape

(1, 114)

In [61]:
a = np.reshape(X_test_c[0],(1,1,-1))
a.shape

(1, 1, 114)

In [63]:
1-new_model.predict(a)

array([[[0.9645596]]], dtype=float32)

In [19]:
# load models
model_quote = tf.keras.models.load_model('my_model_quote.h5')
model_conf = tf.keras.models.load_model('my_model_conf.h5')

# test model on a value
quote_prediction = model_quote.predict(np.reshape(X_test_q[0],(1,1,-1)))
print(quote_prediction[0][0][0])
confidence_prediction = model_conf.predict(np.reshape(X_test_c[0],(1,1,-1)))
print(1-confidence_prediction[0][0][0])

152.34225
0.9645596072077751


In [7]:
X_test_q[0]

array([[0.57      , 0.95988   , 0.99356436, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [188]:
print(y.shape)
print(X.shape)

(5000, 5)
(5000, 114, 5)


In [76]:
y = np.reshape(y,(-1,5))

In [77]:
y.shape

(5000, 5)

In [32]:
df = df.drop(['quote'],axis=1,inplace=True)

In [33]:
df.head()

AttributeError: 'NoneType' object has no attribute 'head'

In [44]:
df1 = pd.read_csv(data_path / file_csv[0],)
df1.head()

,age,city,dist,gender,make,year,model,new_used,quote
0,61,Belleville,10726,m,audi,2017,e-tron,used,352.599790
1,21,Wood Buffalo,9401,m,toyota,2001,camry,used,1723.855245
2,83,Kelowna,9427,m,acura,2016,mdx,used,221.792349
3,28,St. John,21090,m,ford,2003,explorer,used,1723.855245
4,56,Sarnia,7099,m,audi,2018,e-tron,used,354.748787


In [45]:
df1 = df1.drop(['quote'], axis=1, inplace=False)

In [46]:
X = df1.to_numpy()

In [15]:
df1 = pd.get_dummies(df1,drop_first=True)

In [16]:
df1.head()

,age,dist,year,quote,city_Airdrie,city_Ajax,city_Aurora,city_Barrie,city_Belleville,city_Blainville,...,model_mirano,model_r8,model_rsx,model_s,model_sierra,model_terrain,model_x,model_x5,model_yukon,new_used_used
0,61,10726,2017,352.599790,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,21,9401,2001,1723.855245,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,83,9427,2016,221.792349,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,28,21090,2003,1723.855245,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,56,7099,2018,354.748787,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [17]:
X1 = df1.to_numpy()
X1.shape

(5000, 140)